In [2]:
import os
import dotenv


import pandas as pd


from datafin import S3Client

In [3]:
dotenv.load_dotenv()


s3_bucket = os.getenv('PERSONAL_S3_BUCKET_NAME')
aws_access_key =  os.getenv('PERSONAL_AWS_ACCESS_KEY_ID')
aws_secret_key =  os.getenv('PERSONAL_AWS_SECRET_ACCESS_KEY_ID')


polygon_access_key =  os.getenv('POLYGON_AWS_ACCESS_KEY_ID')
polygon_access_key_secret_key =  os.getenv('POLYGON_AWS_SECRET_ACCESS_KEY_ID')

In [4]:
my_s3 = S3Client(
    bucket_name=s3_bucket,
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name='us-east-1'
)

In [5]:
p_s3 = S3Client(
    aws_access_key_id=polygon_access_key,
    aws_secret_access_key=polygon_access_key_secret_key,
    is_polygon=True
)

In [6]:
test = my_s3.get_json(
    'test',
    'test'
)
test

'{"data": {"message": "connection to s3 bucket successful!"}}'

In [7]:
polygon_path_base = 'us_stocks_sip/minute_aggs_v1'
polygon_specific_path = '2025/04'

polygon_combined_path = polygon_path_base + '/' + polygon_specific_path
file_name = '2025-04-24'

data_04_24 = p_s3.get_csv_compressed(
    polygon_combined_path,
    file_name
)

In [9]:
data_04_24['date_time'] = pd.to_datetime(data_04_24['window_start'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None)

In [11]:
import pandas as pd

def fill_missing_intervals(df):
    # Create complete time range for the day
    start_time = pd.Timestamp(df['date_time'].min().date()) + pd.Timedelta(hours=4)
    end_time = pd.Timestamp(df['date_time'].min().date()) + pd.Timedelta(hours=19, minutes=59)
    all_times = pd.date_range(start=start_time, end=end_time, freq='1min')
    
    # Get unique tickers
    tickers = df['ticker'].unique()
    
    # Create a MultiIndex with all combinations of tickers and times
    multi_index = pd.MultiIndex.from_product(
        [tickers, all_times],
        names=['ticker', 'date_time']
    )
    
    # Set the index to ticker and date_time
    df = df.set_index(['ticker', 'date_time'])
    
    # Reindex to include all combinations
    df = df.reindex(multi_index)
    
    # Fill missing values with 0
    numeric_columns = ['volume', 'open', 'close', 'high', 'low', 'transactions']
    df[numeric_columns] = df[numeric_columns].fillna(0)
    
    # Reset index to get back to original format
    df = df.reset_index()
    
    return df

# Apply the function to your dataframe
data_04_24_filled = fill_missing_intervals(data_04_24)

In [20]:
data_04_24_filled_spy = data_04_24_filled[data_04_24_filled['ticker'] == 'SPY'].count()
data_04_24_filled_spy

ticker          960
date_time       960
volume          960
open            960
close           960
high            960
low             960
window_start    900
transactions    960
dtype: int64

In [13]:
data_04_24['ticker'].unique()

array(['A', 'AA', 'AAA', ..., 'ZYME', 'ZYXI', 'ZZZ'],
      shape=(10612,), dtype=object)

In [12]:
from polygon import RESTClient

client = RESTClient("GQn0EJI5jXsETWxkIn5fmVIQTufvfKam")

aggs = []
for a in client.list_aggs(
    "C:EURUSD",
    1,
    "minute",
    "2025-04-24",
    "2025-04-24",
    adjusted="true",
    sort="asc",
    limit=120,
):
    aggs.append(a)

MaxRetryError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/C:EURUSD/range/1/minute/1745488800000/2025-04-24?cursor=bGltaXQ9MTIwJnNvcnQ9YXNj (Caused by ResponseError('too many 429 error responses'))

In [6]:
import pandas as pd

In [10]:
df = pd.DataFrame(tickers)
df['locale'].unique()

array(['us'], dtype=object)